In [ ]:
from IPython.display import HTML
from matplotlib_inline.backend_inline import set_matplotlib_formats

# Use inline backend so CSS scaling works on PNG/SVG outputs
set_matplotlib_formats('png')

HTML("""
<style>
/* =======================
   Matplotlib figure sizing
   ======================= */

/* Images (PNG/retina) */
.jp-RenderedImage img,
.jp-OutputArea-output img {
  width: 100% !important;
  height: auto !important;
  max-width: 100% !important;
  display: block;
  box-sizing: border-box;
}

/* SVG outputs */
.jp-RenderedSVG svg,
.jp-OutputArea-output svg {
  width: 100% !important;
  height: auto !important;
}

/* ipympl / canvas-backed figures (in case %matplotlib widget is used) */
.jupyter-matplotlib canvas.mpl-canvas,
.jp-OutputArea-output canvas.mpl-canvas,
.jp-OutputArea-output canvas {
  width: 100% !important;
  height: auto !important;
  max-width: 100% !important;
}

/* Reset container widths */
.jp-OutputArea-output .matplotlib-figure,
.jp-RenderedHTMLCommon .matplotlib-figure,
.jp-OutputArea-output .output_png {
  width: 100% !important;
}

/* =======================
   ipywidgets styling
   ======================= */

/* Make slider labels bold */
.widget-label {
    font-weight: bold;
}

/* Style ipywidgets buttons */
.jp-OutputArea .widget-button,
button.widget-button {
    font-weight: bold;
    background-color: #1976d2;  /* blue background */
    color: white;               /* white text */
    border: none;
    border-radius: 6px;
    padding: 6px 12px;
    font-size: 14px;
    cursor: pointer;
}

/* Button hover effect */
.jp-OutputArea .widget-button:hover,
button.widget-button:hover {
    background-color: #1259a4;  /* darker blue */
}
</style>
""")

# SAR Collection Modes 🛰️ 📡

## What Do We Want to See? 👀

Every SAR use case starts with a wish list. ✍️  Do we want **ultra-fine detail** in a tiny area 👀🔍, or **good-enough coverage** across a huge swath of Earth 🌍🗺️? How big should the image be 📏, and how sharp should the resolution look ✨?  

Once these wishes are set, the SAR engineer steps in 🧑‍🔧. Their task is to make it real using the radar and antenna system on board 🛰️. That means deciding **how to steer the antenna beam** 🎯 and **how to illuminate the scene** 💡 as the satellite flies along.  

The outcome is a toolbox of clever strategies 🧰 — the **SAR imaging modes** — each one trading off detail, coverage, and efficiency in its own way. ⚖️  

---

## The Dark Room Story 🔦🖼️✍️

Imagine stepping into a pitch-black gallery room 🖼️🌑. Somewhere ahead hangs a massive wall-sized painting 🎨. In your hand is a flashlight 🔦 with a battery that will last exactly **one minute** ⏱️. On a small table you also find paper and a pencil ✍️. You must decide **how** to use that precious minute: where to point the light, how long to dwell, and how to move it as you sketch. That choice is your **imaging mode**.  

With the flashlight in your **left hand** 🔦 and the pencil in your **right** ✏️, you can:  
- Keep the beam locked on one tiny patch 👀,  
- Sweep steadily across the painting ↔️,  
- Or hop between several wider bands of it 📐.  

Each choice changes how much time any patch of the painting is lit — its **dwell time** ⏳ — and therefore how much **detail** you can capture.  

Below, each option mirrors a SAR mode. 🛰️  

---

### Spotlight 🎯 — “Hold still, draw deep detail”

You decide to capture **exquisite detail** from the very **center** of the painting 🎨. Your left hand holds the flashlight perfectly steady 🔦, keeping it **fixed on the same tiny patch** for the entire minute ⏱️. Because that spot stays lit the whole time, your right hand ✏️ can sketch **microscopic detail** — every brushstroke, every crack in the paint 👀.  

**SAR translation:**  
In **Spotlight mode**, the antenna beam is steered to keep illuminating **the same ground patch** while the satellite flies past. The patch remains “under the beam” 📡, giving the **longest synthetic aperture** ➡️📏 and therefore the **finest azimuth resolution** ✨. The trade-off: you only get to image a **small scene** 🗺️.  

> 💡 **Spotlight steering:**  
> The antenna beam can be steered either **mechanically** 🛰️ (by slightly re-orienting the satellite) or **electronically** ⚡ (by adjusting the phases in a phased array). 

> 🎯 **When to choose Spotlight:**  
> For tiny targets, infrastructure, or change detection in compact areas — whenever **detail matters more than coverage**.  


In [ ]:
def build_spotlight_voila_app():
    """
    Voila-friendly SPOTLIGHT radar visualization (minimal controls) with a moving 3D scatter marker.
    - Only adjustable parameter: collection duration (seconds).
    - Beam points to a fixed ground patch (scene center).
    - Growing synthetic-aperture line shows trajectory accrued so far.
    - The platform is labeled by a triangle scatter marker that MOVES along the trajectory.
    """
    import numpy as np
    import matplotlib.pyplot as plt
    from mpl_toolkits.mplot3d import Axes3D  # noqa: F401
    from mpl_toolkits.mplot3d.art3d import Poly3DCollection
    from ipywidgets import FloatSlider, HBox, VBox, Play, IntSlider, jslink, Button, Layout, Output, Label
    from IPython.display import display, clear_output

    # ---------- Constants (no UI) ----------
    SATELLITE_VELOCITY_KMS = 7.5      # km/s
    ALTITUDE_KM            = 500.0    # km
    INCIDENCE_DEG          = 30.0     # degrees
    BEAM_HALF_ANGLE_DEG    = 1.0      # degrees
    FPS                    = 12       # frames per second for the animation
    MIN_FRAMES             = 40
    MAX_FRAMES             = 600

    # ---------- Single control ----------
    collection_time = FloatSlider(
        value=10.0, min=2.0, max=30.0, step=1.0,
        description="Collection Time (s)", continuous_update=False
    )
    collection_time.style.description_width = 'initial'

    # Timeline controls
    play = Play(interval=80, value=0, min=0, max=99, step=1)
    frame_slider = IntSlider(value=0, min=0, max=99, step=1, description="Frame", continuous_update=True)
    jslink((play, 'value'), (frame_slider, 'value'))

    reset_btn = Button(description="Reset", layout=Layout(width='90px'))
    recenter_btn = Button(description="Recenter", layout=Layout(width='110px'))

    out = Output()

    # ---------- State ----------
    state = dict(
        x_positions=None, y_positions=None, z_positions=None,
        artists={}, fig=None, theta=np.linspace(0, 2*np.pi, 120),
        frames=100
    )

    ground_point = (0.0, 0.0, 0.0)

    # ---------- Helpers ----------
    def _compute_frames():
        frames = int(collection_time.value * FPS)
        return int(np.clip(frames, MIN_FRAMES, MAX_FRAMES))

    def _compute_geometry():
        # Platform flies along +X; cross-track Y by incidence angle at given altitude (flat-Earth sketch)
        aperture_length = SATELLITE_VELOCITY_KMS * collection_time.value  # km
        max_aperture_length = SATELLITE_VELOCITY_KMS * 30.0  # km (max collection time)
        x_start, x_end = -aperture_length / 2.0, aperture_length / 2.0
        xs = np.linspace(x_start, x_end, state['frames'])
        y_pos = ALTITUDE_KM * np.tan(np.radians(INCIDENCE_DEG))
        ys = np.full_like(xs, y_pos)
        zs = np.full_like(xs, ALTITUDE_KM)

        state['x_positions'], state['y_positions'], state['z_positions'] = xs, ys, zs

        # Axes limits with margins
        x_start_max = -max_aperture_length / 2.0
        x_end_max = max_aperture_length / 2.0
        mx = 0.1 * (x_end_max - x_start_max + 1.0)
        xlim = (x_start_max - mx, x_end_max + mx)
        ylim = (-0.2 * (abs(y_pos) + 1.0), y_pos + abs(y_pos) * 0.2 + 10.0)
        zlim = (0.0, ALTITUDE_KM * 1.1 + 10.0)
        return xlim, ylim, zlim

    # ---------- Plot init ----------
    def _init_plot():
        xlim, ylim, zlim = _compute_geometry()

        fig = plt.figure(figsize=(9, 6))  # noqa: E999 (keep size as in your original)
        ax = fig.add_subplot(111, projection='3d')
        ax.set_title("SPOTLIGHT imaging mode")
        ax.set_xlabel("X (km) — azimuth")
        ax.set_ylabel("Y (km) — ground range")
        ax.set_zlabel("Z (km) — altitude")
        ax.set_xlim(*xlim); ax.set_ylim(*ylim); ax.set_zlim(*zlim)

        # Static full flight path (light reference)
        flight_path = ax.plot(state['x_positions'], state['y_positions'], state['z_positions'],
                              lw=1.0, color='0.8', alpha=0.8)[0]

        # Scene center
        gp_dot = ax.plot([ground_point[0]], [ground_point[1]], [ground_point[2]],
                         marker='o', markersize=6, color='k', label="Scene Center")[0]

        # Dynamic artists
        los_line   = ax.plot([], [], [], linestyle='--', lw=1.5, color='red', label="Beam Center")[0]
        beam_left  = ax.plot([], [], [], linestyle='--', lw=1.0, color='black', label="Beam Edge")[0]
        beam_right = ax.plot([], [], [], linestyle='--', lw=1.0, color='black')[0]

        # Growing synthetic aperture (trajectory accrued so far)
        aperture_line, = ax.plot([], [], [], lw=3.0, color='blue', label="Synthetic Aperture (trajectory)")

        # Elliptical ground footprint (z=0)
        footprint = Poly3DCollection([[(0,0,0)]], alpha=0.5, facecolor='yellow', edgecolor='black')
        ax.add_collection3d(footprint)

        # --- Satellite scatter marker (replaces PNG) ---
        satellite_marker = ax.scatter(
            [state['x_positions'][0]],
            [state['y_positions'][0]],
            [state['z_positions'][0]],
            marker='^',            # triangle looks "satellite-ish"
            s=120,                 # size
            color='orange',
            edgecolors='k',
            linewidths=0.8,
            label='Satellite'
        )

        ax.legend(
            loc='upper left',
            bbox_to_anchor=(1.05, 1),   # move outside to the right
            borderaxespad=0.,
            fontsize=9
        )
        
        state['fig'] = fig
        state['artists'] = dict(
            ax=ax, flight_path=flight_path, gp_dot=gp_dot,
            los_line=los_line, beam_left=beam_left, beam_right=beam_right,
            aperture_line=aperture_line, footprint=footprint, gp=ground_point,
            satellite_marker=satellite_marker
        )

    # ---------- Render (Voila-safe) ----------
    def _render():
        with out:
            clear_output(wait=True)
            display(state['fig'])
            plt.close(state['fig'])

    # ---------- Frame update ----------
    def _update_frame(k):
        if state['x_positions'] is None:
            return

        xk = state['x_positions'][k]
        yk = state['y_positions'][k]
        zk = state['z_positions'][k]

        A = state['artists']; ax = A['ax']; gp = A['gp']

        # LOS to ground point
        A['los_line'].set_data([xk, gp[0]], [yk, gp[1]])
        A['los_line'].set_3d_properties([zk, gp[2]])

        # Beam spread and edges
        rng = np.sqrt((xk-gp[0])**2 + (yk-gp[1])**2 + (zk-gp[2])**2)
        hw = np.tan(np.radians(BEAM_HALF_ANGLE_DEG)) * rng
        A['beam_left'].set_data([xk, gp[0] - hw], [yk, gp[1]])
        A['beam_left'].set_3d_properties([zk, gp[2]])
        A['beam_right'].set_data([xk, gp[0] + hw], [yk, gp[1]])
        A['beam_right'].set_3d_properties([zk, gp[2]])

        # Elliptical footprint (elongated in ground-range by 1/cos(inc))
        cosi = max(np.cos(np.radians(INCIDENCE_DEG)), 1e-6)
        a_az = hw
        a_rg = hw / (0.2 * cosi)
        th = state['theta']
        xc = gp[0] + a_az * np.cos(th)
        yc = gp[1] + a_rg * np.sin(th)
        zc = np.zeros_like(th)
        A['footprint'].set_verts([list(zip(xc, yc, zc))])

        # Growing synthetic aperture (trajectory so far)
        A['aperture_line'].set_data(state['x_positions'][:k+1], state['y_positions'][:k+1])
        A['aperture_line'].set_3d_properties(state['z_positions'][:k+1])

        # --- Move the satellite scatter marker ---
        # For 3D scatter, updating uses the protected attribute _offsets3d:
        A['satellite_marker']._offsets3d = ([xk], [yk], [zk])

        _render()

    # ---------- Rebuild & callbacks ----------
    def _rebuild_scene(*_):
        state['frames'] = _compute_frames()
        play.min = 0; play.max = state['frames'] - 1
        frame_slider.min = 0; frame_slider.max = state['frames'] - 1
        frame_slider.value = 0
        _init_plot()
        _update_frame(0)

    def _on_reset(_):
        frame_slider.value = 0
        _update_frame(0)

    def _on_recenter(_):
        xlim, ylim, zlim = _compute_geometry()
        ax = state['artists']['ax']
        ax.set_xlim(*xlim); ax.set_ylim(*ylim); ax.set_zlim(*zlim)
        _render()

    frame_slider.observe(lambda ch: _update_frame(ch['new']), names='value')
    reset_btn.on_click(_on_reset)
    recenter_btn.on_click(_on_recenter)
    collection_time.observe(_rebuild_scene, names='value')

    # Build once
    _rebuild_scene()

    # ---------- Layout ----------
    header = Label("SPOTLIGHT: Fixed scene center — moving marker labels the platform.")
    controls = HBox([collection_time, reset_btn, recenter_btn])
    timeline = HBox([play, frame_slider])

    return VBox([header, controls, timeline, out])


# Build & display
spotlight_app = build_spotlight_voila_app()
display(spotlight_app)

### Stripmap 📜 — “Sweep steadily, draw a clean strip”  

Now you want more context. Instead of freezing the beam on one spot, you set the flashlight to a **fixed tilt** 🔦 and sweep it smoothly left-to-right ↔️ across the wall. Each patch of the painting is lit for a **moderate amount of time** ⏱️ — less than Spotlight, but the same for every point along the sweep. The result on your sketch ✏️ is a **long, continuous band**: steady, uniform, and with **good (but not extreme) detail**.  

**SAR translation:**  
In **Stripmap mode**, the antenna beam points in a fixed direction 📡 while the satellite flies forward 🛰️, continuously illuminating a **ground strip**. Each ground point remains in the beam only for the time set by the antenna’s **beamwidth** 📏, so the illumination time is shorter than in Spotlight. The result:  
- **Moderate azimuth resolution** (classically ~½ the antenna length along-track)  
- A **clean, continuous swath** 🗺️  

> 🎯 **When to choose Stripmap:**  
> Corridor mapping and monitoring where you want **steady coverage** with **solid, consistent detail**.  

> 🔧 **Myth-buster (Stripmap):**  
> Counter-intuitively, making the **azimuth antenna smaller** 📐 actually improves **azimuth resolution** in stripmap SAR!  
> For broadside stripmap, azimuth resolution ≈ ½ the antenna length.  
> 👉 So **shorter antenna → smaller resolution cell → finer detail**.  
> ⚠️ **But trade-offs:** a smaller antenna reduces **gain and SNR** 📉 and forces a **higher pulse repetiton frequency (PRF)** to avoid azimuth ambiguities. There are no free lunches in radar system design 🥗.  

> 🆚 **Contrast with real-aperture radar (RAR):**  
> In RAR, **longer antenna → narrower beam → finer resolution**.  
> In Stripmap SAR, this flips: resolution comes from combining echoes along the synthetic aperture, not just the raw beamwidth.  


In [ ]:
def build_stripmap_voila_app():
    """
    Voila-friendly STRIPMAP radar visualization (real-aperture, side-looking).
    - Single adjustable parameter: collection duration (seconds).
    - Beam center always perpendicular to the flight path (side-looking).
    - Ground intercept (beam center) sweeps a strip along X at Y=0.
    - Shaded ground band shows the imaged strip; gold ellipse is instantaneous footprint.
    - Growing synthetic-aperture line shows trajectory accrued so far.
    Rendering uses ipywidgets.Output + manual redraw so Voila updates reliably.
    """
    import numpy as np
    import matplotlib.pyplot as plt
    from mpl_toolkits.mplot3d import Axes3D  # noqa: F401
    from mpl_toolkits.mplot3d.art3d import Poly3DCollection
    from ipywidgets import FloatSlider, IntSlider, HBox, VBox, Play, jslink, Button, Layout, Output, Label
    from IPython.display import display, clear_output

    # ---------- Constants (no UI) ----------
    SATELLITE_VELOCITY_KMS = 7.5      # km/s
    ALTITUDE_KM            = 500.0    # km
    INCIDENCE_DEG          = 30.0     # deg from vertical
    BEAM_HALF_ANGLE_DEG    = 1.0      # deg (main-lobe half-angle)
    FPS                    = 12       # frames per second for the animation
    MIN_FRAMES             = 40
    MAX_FRAMES             = 600
    # Visual stretching factor for ellipse range axis (keeps it readable)
    K_ELLIPTICITY          = 0.2

    # ---------- Single control ----------
    collection_time = FloatSlider(
        value=10.0, min=2.0, max=40.0, step=1.0,
        description="Collection Time (s)", continuous_update=False
    )
    collection_time.style.description_width = 'initial'

    # Timeline controls
    play = Play(interval=80, value=0, min=0, max=99, step=1)
    frame_slider = IntSlider(value=0, min=0, max=99, step=1, description="Frame", continuous_update=True)
    jslink((play, 'value'), (frame_slider, 'value'))

    reset_btn = Button(description="Reset", layout=Layout(width='90px'))
    recenter_btn = Button(description="Recenter", layout=Layout(width='110px'))

    out = Output()

    # ---------- State ----------
    state = dict(
        x_positions=None, y_positions=None, z_positions=None,
        artists={}, fig=None, theta=np.linspace(0, 2*np.pi, 120),
        frames=100
    )

    # ---------- Helpers ----------
    def _compute_frames():
        frames = int(collection_time.value * FPS)
        return int(np.clip(frames, MIN_FRAMES, MAX_FRAMES))

    def _compute_geometry():
        """
        STRIPMAP geometry:
        - Platform flies along +X at a fixed cross-track Y (set by incidence) and fixed altitude Z.
        - Beam center looks to ground at Y=0 (perpendicular to track).
        - Slant range is constant along track in this simple sketch.
        """
        frames = state['frames']
        aperture_length = SATELLITE_VELOCITY_KMS * collection_time.value  # km
        x_start, x_end = -aperture_length / 2.0, aperture_length / 2.0
        xs = np.linspace(x_start, x_end, frames)

        # Platform cross-track (positive Y) given incidence at altitude (flat-Earth sketch)
        y_pos = ALTITUDE_KM * np.tan(np.radians(INCIDENCE_DEG))
        ys = np.full_like(xs, y_pos)
        zs = np.full_like(xs, ALTITUDE_KM)

        state['x_positions'], state['y_positions'], state['z_positions'] = xs, ys, zs

        # Ground-plane beam footprint parameters (constant here)
        rng  = np.sqrt(y_pos**2 + ALTITUDE_KM**2)                         # slant range (km)
        hw   = np.tan(np.radians(BEAM_HALF_ANGLE_DEG)) * rng              # half-width at slant range (km)
        cosi = max(np.cos(np.radians(INCIDENCE_DEG)), 1e-6)
        a_az = hw                                                         # azimuth semi-axis (X)
        a_rg = hw / (K_ELLIPTICITY * cosi)                                # range semi-axis (Y), visually elongated

        # Shaded strip half-width around Y=0 (use a_rg to represent imaged band)
        strip_half_width = a_rg

        # Axes limits with margins
        mx = 0.1 * (x_end - x_start + 1.0)
        max_length = SATELLITE_VELOCITY_KMS * 50.0  # km (max collection time)
        xlim = (-max_length/2 - mx, max_length/2 + mx)

        # Y-limits must include platform Y, strip band (±strip_half_width), and 0
        y_candidates = np.array([-strip_half_width, strip_half_width, y_pos, 0.0], dtype=float)
        ymin = float(np.min(y_candidates))
        ymax = float(np.max(y_candidates))
        ymargin = 0.1 * (ymax - ymin + 1.0)
        ylim = (ymin - ymargin, ymax + ymargin)

        zlim = (0.0, ALTITUDE_KM * 1.1 + 10.0)

        return (x_start, x_end), xlim, ylim, zlim, a_az, a_rg, strip_half_width

    # ---------- Plot init ----------
    def _init_plot():
        (_, _), xlim, ylim, zlim, a_az, a_rg, strip_hw = _compute_geometry()

        fig = plt.figure(figsize=(9, 6))
        ax = fig.add_subplot(111, projection='3d')
        ax.set_title("STRIPMAP imaging mode")
        ax.set_xlabel("X (km) — azimuth")
        ax.set_ylabel("Y (km) — ground range")
        ax.set_zlabel("Z (km) — altitude")

        xs = state['x_positions']; ys = state['y_positions']; zs = state['z_positions']
        x_start, x_end = xs[0], xs[-1]

        # --- Shaded ground strip (image area on z=0) ---
        strip_poly = [
            (x_start, -strip_hw, 0.0),
            (x_end,   -strip_hw, 0.0),
            (x_end,    strip_hw, 0.0),
            (x_start,  strip_hw, 0.0),
        ]
        strip_patch = Poly3DCollection([strip_poly], alpha=0.5, facecolor='lightgreen', edgecolor='none')
        ax.add_collection3d(strip_patch)

        # Static references
        flight_path = ax.plot(xs, ys, zs, lw=1.0, color='0.8', alpha=0.9, label="Full Flight Path")[0]
        ground_centerline = ax.plot(xs, np.zeros_like(xs), np.zeros_like(xs), lw=1.0, color='0.7', label="Ground Strip Center")[0]

        # Dynamic artists
        sat_marker = ax.scatter([xs[0]], [ys[0]], [zs[0]],
                                marker='^', s=120, color='orange', edgecolors='k', linewidths=0.8,
                                label='Radar Platform')
        los_line   = ax.plot([], [], [], linestyle='--', lw=1.5, color='red',  label="Beam Center")[0]
        beam_left  = ax.plot([], [], [], linestyle='--', lw=1.0, color='black', label="Beam Edge")[0]
        beam_right = ax.plot([], [], [], linestyle='--', lw=1.0, color='black')[0]

        # Growing synthetic aperture (trajectory accrued so far)
        aperture_line, = ax.plot([], [], [], lw=3.0, color='blue', label="Synthetic Aperture (trajectory)")

        # Instantaneous elliptical footprint on ground (z=0)
        footprint = Poly3DCollection([[(0,0,0)]], alpha=0.6, facecolor='gold', edgecolor='black')
        ax.add_collection3d(footprint)

        # Legend (moved outside to the right)
        ax.legend(
            [sat_marker, aperture_line, los_line, beam_left, footprint, strip_patch, ground_centerline],
            ["Radar Platform", "Aperture (visited track)", "Beam Center", "Beam Edge",
            "Instant Footprint", "Imaged Strip (shaded)", "Strip Centerline"],
            loc='upper left',
            bbox_to_anchor=(1.05, 1),   # move to the right outside the axes
            borderaxespad=0.,
            fontsize=9
        )

        # --- Lock axes & aspect AFTER artists are added (prevents drift and distortion) ---
        ax.set_autoscale_on(False)
        ax.set_xlim(*xlim); ax.set_ylim(*ylim); ax.set_zlim(*zlim)
        xspan = xlim[1] - xlim[0]
        yspan = ylim[1] - ylim[0]
        zspan = zlim[1] - zlim[0]
        ax.set_box_aspect((xspan, yspan, zspan))

        state['fig'] = fig
        state['artists'] = dict(
            ax=ax,
            flight_path=flight_path, ground_centerline=ground_centerline,
            strip_patch=strip_patch,
            sat_marker=sat_marker, los_line=los_line,
            beam_left=beam_left, beam_right=beam_right,
            aperture_line=aperture_line, footprint=footprint,
            a_az=a_az, a_rg=a_rg,
            xlim=xlim, ylim=ylim, zlim=zlim
        )

    # ---------- Render helper (crucial for Voila) ----------
    def _render():
        with out:
            clear_output(wait=True)
            display(state['fig'])
            plt.close(state['fig'])  # avoid extra static figure below

    # ---------- Frame update ----------
    def _update_frame(k):
        if state['x_positions'] is None:
            return

        xs = state['x_positions']; ys = state['y_positions']; zs = state['z_positions']
        xk, yk, zk = xs[k], ys[k], zs[k]
        gp_x, gp_y, gp_z = xk, 0.0, 0.0  # ground intercept directly to the side

        A = state['artists']; ax = A['ax']
        a_az = A['a_az']; a_rg = A['a_rg']

        # Move platform marker
        A['sat_marker']._offsets3d = ([xk], [yk], [zk])

        # LOS (platform -> ground intercept)
        A['los_line'].set_data([xk, gp_x], [yk, gp_y])
        A['los_line'].set_3d_properties([zk, gp_z])

        # Beam edges at ground (left/right in azimuth about intercept)
        A['beam_left'].set_data([xk, gp_x - a_az], [yk, gp_y])
        A['beam_left'].set_3d_properties([zk, gp_z])
        A['beam_right'].set_data([xk, gp_x + a_az], [yk, gp_y])
        A['beam_right'].set_3d_properties([zk, gp_z])

        # Instantaneous ground footprint (ellipse on z=0)
        th = state['theta']
        xc = gp_x + a_az * np.cos(th)
        yc = gp_y + a_rg * np.sin(th)
        zc = np.zeros_like(th)
        A['footprint'].set_verts([list(zip(xc, yc, zc))])

        # Growing synthetic aperture (trajectory so far)
        A['aperture_line'].set_data(xs[:k+1], ys[:k+1])
        A['aperture_line'].set_3d_properties(zs[:k+1])

        _render()

    # ---------- Rebuild & callbacks ----------
    def _rebuild_scene(*_):
        state['frames'] = _compute_frames()
        play.min = 0; play.max = state['frames'] - 1
        frame_slider.min = 0; frame_slider.max = state['frames'] - 1
        frame_slider.value = 0
        _init_plot()
        _update_frame(0)

    def _on_reset(_):
        frame_slider.value = 0
        _update_frame(0)

    def _on_recenter(_):
        # Recompute geometry & limits, then reapply locking
        (_, _), xlim, ylim, zlim, *_ = _compute_geometry()
        ax = state['artists']['ax']
        ax.set_autoscale_on(False)
        ax.set_xlim(*xlim); ax.set_ylim(*ylim); ax.set_zlim(*zlim)
        xspan = xlim[1] - xlim[0]
        yspan = ylim[1] - ylim[0]
        zspan = zlim[1] - zlim[0]
        ax.set_box_aspect((xspan, yspan, zspan))
        _render()

    frame_slider.observe(lambda ch: _update_frame(ch['new']), names='value')
    reset_btn.on_click(_on_reset)
    recenter_btn.on_click(_on_recenter)
    collection_time.observe(_rebuild_scene, names='value')

    # Build once
    _rebuild_scene()

    # ---------- Layout ----------
    header   = Label("STRIPMAP: Side-looking real-aperture — shaded image strip & moving platform.")
    controls = HBox([collection_time, reset_btn, recenter_btn])
    timeline = HBox([play, frame_slider])

    return VBox([header, controls, timeline, out])


# Build & display
stripmap_app = build_stripmap_voila_app()
display(stripmap_app)

### Sliding Spotlight 🎢 — “Glide the beam, balance detail and coverage”  

What if you don’t want to stay locked on one tiny patch 👀 like in **Spotlight**, but you also don’t want to sweep past at full speed ↔️ like in **Stripmap**?  In **Sliding Spotlight**, the flashlight beam 🔦 is steered so that it **follows the scene**, but instead of holding on a single patch, it **slowly glides forward along the wall**.  Your sketch ✏️ captures **more area than pure Spotlight**, yet with **finer detail than Stripmap**. The trade-off: each patch gets less dwell time ⏱️ than Spotlight, so the **resolution is coarser** — but in return you gain extra coverage. ⚖️  

**SAR translation:**  
In **Sliding Spotlight mode**, the antenna beam is gradually steered in azimuth while the platform moves.  
- The beam lingers longer than in Stripmap ➡️ better resolution  
- But it doesn’t stay fixed like in Spotlight ➡️ more coverage  
- The result: a **compromise between resolution and swath width** ⚖️.  

> 🎯 **When to choose Sliding Spotlight:**  
> Situations where you want a balance — **sharper detail than Stripmap**, but **more area than Spotlight**.  

Adjust the **Glide Fraction** slider to explore Sliding Spotlight: **0 = pure Spotlight**, **1 = pure Stripmap**, and values in between show the smooth transition.  

In [ ]:
def build_sliding_spotlight_voila_app():
    """
    Voila-friendly SLIDING SPOTLIGHT radar visualization.
    - Two controls: collection duration (s) and glide fraction (0=Spotlight, 1=Stripmap).
    - Beam center slowly glides in azimuth: gp_x = alpha * x_platform + (1 - alpha) * 0.
    - Growing synthetic-aperture line shows trajectory; moving triangle is the platform.
    - Ground footprint ellipse updates each frame; shaded swath shows the area visited so far.
    """
    import numpy as np
    import matplotlib.pyplot as plt
    from mpl_toolkits.mplot3d import Axes3D  # noqa: F401
    from mpl_toolkits.mplot3d.art3d import Poly3DCollection
    from ipywidgets import FloatSlider, IntSlider, HBox, VBox, Play, jslink, Button, Layout, Output, Label
    from IPython.display import display, clear_output

    # ---------- Constants ----------
    SATELLITE_VELOCITY_KMS = 7.5      # km/s
    ALTITUDE_KM            = 500.0    # km
    INCIDENCE_DEG          = 30.0     # deg from vertical
    BEAM_HALF_ANGLE_DEG    = 1.0      # deg (main-lobe half-angle)
    FPS                    = 12       # frames per second
    MIN_FRAMES             = 40
    MAX_FRAMES             = 600
    K_ELLIPTICITY          = 0.2      # visual stretch for ground-range axis (readability)

    # ---------- Controls ----------
    collection_time = FloatSlider(
        value=10.0, min=2.0, max=40.0, step=1.0,
        description="Collect T (s)", continuous_update=False
    )
    collection_time.style.description_width = 'initial'
    glide_fraction = FloatSlider(
        value=0.4, min=0.0, max=1.0, step=0.05,
        description="Glide Fraction (0–1)", continuous_update=False
    )
    glide_fraction.style.description_width = 'initial'

    play = Play(interval=80, value=0, min=0, max=99, step=1)
    frame_slider = IntSlider(value=0, min=0, max=99, step=1, description="Frame", continuous_update=True)
    jslink((play, 'value'), (frame_slider, 'value'))

    reset_btn = Button(description="Reset", layout=Layout(width='90px'))
    recenter_btn = Button(description="Recenter", layout=Layout(width='110px'))

    out = Output()

    # ---------- State ----------
    state = dict(
        x_positions=None, y_positions=None, z_positions=None,
        artists={}, fig=None, theta=np.linspace(0, 2*np.pi, 120),
        frames=100,
        gp_x_hist=[]  # ground intercept history (for shaded swath)
    )

    # ---------- Helpers ----------
    def _compute_frames():
        frames = int(collection_time.value * FPS)
        return int(np.clip(frames, MIN_FRAMES, MAX_FRAMES))

    def _platform_track():
        """Platform flies along +X at fixed cross-track Y (set by incidence) and fixed altitude Z."""
        frames = state['frames']
        aperture_length = SATELLITE_VELOCITY_KMS * collection_time.value  # km
        xs = np.linspace(-aperture_length/2.0, aperture_length/2.0, frames)
        y_pos = ALTITUDE_KM * np.tan(np.radians(INCIDENCE_DEG))
        ys = np.full_like(xs, y_pos)
        zs = np.full_like(xs, ALTITUDE_KM)
        return xs, ys, zs

    def _axes_limits(xs, ys, zs, strip_like_halfwidth):
        """Compute stable axes with margins that contain platform, swath, and ground."""
        x_start, x_end = xs[0], xs[-1]
        mx = 0.1 * (x_end - x_start + 1.0)
        max_length = SATELLITE_VELOCITY_KMS * 50.0
        xlim = (-max_length/2 - mx, max_length/2 + mx)

        y_candidates = np.array([ys[0], 0.0, strip_like_halfwidth, -strip_like_halfwidth], dtype=float)
        ymin = float(np.min(y_candidates)); ymax = float(np.max(y_candidates))
        ymargin = 0.1 * (ymax - ymin + 1.0)
        ylim = (ymin - ymargin, ymax + ymargin)

        zlim = (0.0, ALTITUDE_KM * 1.1 + 10.0)
        return xlim, ylim, zlim

    # ---------- Build geometry & persistent values ----------
    def _compute_geometry_basics():
        xs, ys, zs = _platform_track()
        state['x_positions'], state['y_positions'], state['z_positions'] = xs, ys, zs

        # Use mid-track geometry to get a representative footprint size for initial limits
        y_pos = ys[0]
        rng  = np.sqrt(y_pos**2 + ALTITUDE_KM**2)
        hw   = np.tan(np.radians(BEAM_HALF_ANGLE_DEG)) * rng        # half-width at slant range
        cosi = max(np.cos(np.radians(INCIDENCE_DEG)), 1e-6)
        a_az_est = hw
        a_rg_est = hw / (K_ELLIPTICITY * cosi)
        strip_like_halfwidth = a_rg_est  # for initial ylim

        xlim, ylim, zlim = _axes_limits(xs, ys, zs, strip_like_halfwidth)
        return xlim, ylim, zlim

    # ---------- Plot init ----------
    def _init_plot():
        xlim, ylim, zlim = _compute_geometry_basics()
        xs, ys, zs = state['x_positions'], state['y_positions'], state['z_positions']
        x_start, x_end = xs[0], xs[-1]

        fig = plt.figure(figsize=(9, 6))
        ax = fig.add_subplot(111, projection='3d')
        ax.set_title("SLIDING SPOTLIGHT imaging mode")
        ax.set_xlabel("X (km) — azimuth")
        ax.set_ylabel("Y (km) — ground range")
        ax.set_zlabel("Z (km) — altitude")

        # Static references
        flight_path = ax.plot(xs, ys, zs, lw=1.0, color='0.8', alpha=0.9, label="Full Flight Path")[0]
        ground_centerline = ax.plot([x_start, x_end], [0.0, 0.0], [0.0, 0.0], lw=1.0, color='0.7',
                                    label="Ground Centerline")[0]

        # Dynamic artists
        sat_marker = ax.scatter([xs[0]], [ys[0]], [zs[0]],
                                marker='^', s=120, color='orange', edgecolors='k', linewidths=0.8,
                                label='Radar Platform')

        los_line   = ax.plot([], [], [], linestyle='--', lw=1.5, color='red',  label="Beam Center")[0]
        beam_left  = ax.plot([], [], [], linestyle='--', lw=1.0, color='black', label="Beam Edge")[0]
        beam_right = ax.plot([], [], [], linestyle='--', lw=1.0, color='black')[0]

        # Growing synthetic aperture (trajectory so far)
        aperture_line, = ax.plot([], [], [], lw=3.0, color='blue', label="Synthetic Aperture (trajectory)")

        # Instantaneous footprint on ground (z=0)
        footprint = Poly3DCollection([[(0,0,0)]], alpha=0.6, facecolor='gold', edgecolor='black')
        ax.add_collection3d(footprint)

        # Shaded swath (area visited by the sliding beam on z=0)
        swath_poly = Poly3DCollection([[(0,0,0)]], alpha=0.75, facecolor='lightgreen', edgecolor='none')
        ax.add_collection3d(swath_poly)

        # Legend
        ax.legend(
            [sat_marker, aperture_line, los_line, beam_left, footprint, swath_poly, ground_centerline],
            ["Radar Platform", "Aperture (visited track)", "Beam Center", "Beam Edge",
             "Instant Footprint", "Visited Swath (sliding)", "Ground Centerline"],
            loc='upper left',
            bbox_to_anchor=(1.05, 1),
            borderaxespad=0.,
            fontsize=9
        )

        # Lock axes & aspect
        ax.set_autoscale_on(False)
        ax.set_xlim(*xlim); ax.set_ylim(*ylim); ax.set_zlim(*zlim)
        xspan = xlim[1] - xlim[0]; yspan = ylim[1] - ylim[0]; zspan = zlim[1] - zlim[0]
        ax.set_box_aspect((xspan, yspan, zspan))

        state['fig'] = fig
        state['artists'] = dict(
            ax=ax,
            flight_path=flight_path, ground_centerline=ground_centerline,
            sat_marker=sat_marker, los_line=los_line,
            beam_left=beam_left, beam_right=beam_right,
            aperture_line=aperture_line, footprint=footprint,
            swath_poly=swath_poly
        )

        # Reset swath history
        state['gp_x_hist'] = []

    # ---------- Render helper (Voila-safe) ----------
    def _render():
        with out:
            clear_output(wait=True)
            display(state['fig'])
            plt.close(state['fig'])

    # ---------- Frame update ----------
    def _update_frame(k):
        if state['x_positions'] is None:
            return

        xs = state['x_positions']; ys = state['y_positions']; zs = state['z_positions']
        xk, yk, zk = xs[k], ys[k], zs[k]

        A = state['artists']; ax = A['ax']

        # Ground intercept (beam center) — interpolate between Spotlight (0) and Stripmap (1)
        alpha = float(glide_fraction.value)
        gp_x = alpha * xk + (1.0 - alpha) * 0.0
        gp_y, gp_z = 0.0, 0.0

        # Move platform marker
        A['sat_marker']._offsets3d = ([xk], [yk], [zk])

        # LOS (platform -> ground intercept)
        A['los_line'].set_data([xk, gp_x], [yk, gp_y])
        A['los_line'].set_3d_properties([zk, gp_z])

        # Beam geometry at ground (ellipse) based on instantaneous slant range
        rng  = np.sqrt((yk - gp_y)**2 + (zk - gp_z)**2 + (xk - gp_x)**2)   # slant distance to intercept
        hw   = np.tan(np.radians(BEAM_HALF_ANGLE_DEG)) * rng
        cosi = max(np.cos(np.radians(INCIDENCE_DEG)), 1e-6)
        a_az = hw
        a_rg = hw / (K_ELLIPTICITY * cosi)

        # Beam edges (left/right in azimuth at ground)
        A['beam_left'].set_data([xk, gp_x - a_az], [yk, gp_y])
        A['beam_left'].set_3d_properties([zk, gp_z])
        A['beam_right'].set_data([xk, gp_x + a_az], [yk, gp_y])
        A['beam_right'].set_3d_properties([zk, gp_z])

        # Instantaneous ground footprint (ellipse on z=0 centered at gp_x,0)
        th = state['theta']
        xc = gp_x + a_az * np.cos(th)
        yc = gp_y + a_rg * np.sin(th)
        zc = np.zeros_like(th)
        A['footprint'].set_verts([list(zip(xc, yc, zc))])

        # Growing synthetic aperture (trajectory so far)
        A['aperture_line'].set_data(xs[:k+1], ys[:k+1])
        A['aperture_line'].set_3d_properties(zs[:k+1])

        # Update swath polygon from history of gp_x (visited beam centers)
        state['gp_x_hist'].append(gp_x)
        xmin = np.min(state['gp_x_hist']); xmax = np.max(state['gp_x_hist'])
        swath_poly = [
            (xmin, -a_rg, 0.0),
            (xmax, -a_rg, 0.0),
            (xmax,  a_rg, 0.0),
            (xmin,  a_rg, 0.0),
        ]
        A['swath_poly'].set_verts([swath_poly])

        _render()

    # ---------- Rebuild & callbacks ----------
    def _rebuild_scene(*_):
        state['frames'] = _compute_frames()
        play.min = 0; play.max = state['frames'] - 1
        frame_slider.min = 0; frame_slider.max = state['frames'] - 1
        frame_slider.value = 0
        _init_plot()
        _update_frame(0)

    def _on_reset(_):
        frame_slider.value = 0
        _update_frame(0)

    def _on_recenter(_):
        xlim, ylim, zlim = _compute_geometry_basics()
        ax = state['artists']['ax']
        ax.set_autoscale_on(False)
        ax.set_xlim(*xlim); ax.set_ylim(*ylim); ax.set_zlim(*zlim)
        xspan = xlim[1] - xlim[0]; yspan = ylim[1] - ylim[0]; zspan = zlim[1] - zlim[0]
        ax.set_box_aspect((xspan, yspan, zspan))
        _render()

    frame_slider.observe(lambda ch: _update_frame(ch['new']), names='value')
    reset_btn.on_click(_on_reset)
    recenter_btn.on_click(_on_recenter)
    collection_time.observe(_rebuild_scene, names='value')
    glide_fraction.observe(_rebuild_scene, names='value')

    # Build once
    _rebuild_scene()

    # ---------- Layout ----------
    header   = Label("SLIDING SPOTLIGHT: Beam glides in azimuth — Glide Fraction=0 (Spotlight), Glide Fraction=1 (Stripmap).")
    controls = HBox([collection_time, glide_fraction, reset_btn, recenter_btn])
    timeline = HBox([play, frame_slider])

    return VBox([header, controls, timeline, out])


# Build & display in a notebook cell:
sliding_spot_app = build_sliding_spotlight_voila_app()
display(sliding_spot_app)

### ScanSAR 🗺️🧩 — “Hop around, build the big picture”  

Now you want to see the **entire painting** 🎨. You mentally divide the wall into several **horizontal bands** 📐. Instead of holding steady or sweeping smoothly, you **flick the flashlight** 🔦 between band A, then B, then C, then D… and repeat. Your sketch ✏️ becomes a **mosaic** 🧩: a huge area captured, but each band only gets **quick flashes of light** ⏱️ before you jump away. That means any one patch receives the **shortest illumination time** and therefore the **least detail** of all three modes.  

**SAR translation:**  
In **ScanSAR**, the antenna beam is rapidly **switched between multiple sub-swaths** 📡 (in the range/elevation direction, typically using electronic beam steering ⚡).  
- Each sub-swath only gets **short bursts** of illumination,  
- Illumination time per patch is therefore **shortest**,  
- Azimuth resolution is the **coarsest** of the three modes,  
- But the reward: **massive coverage** 🌍.  

> 🎯 **When to choose ScanSAR:**  
> Wide-area situational awareness — disaster mapping 🌪️, sea ice monitoring 🧊, or large-scale land cover 🌱 — whenever **area matters more than detail**.  


In [ ]:
def build_topsar_voila_app():
    """
    Voila-friendly TOPSAR / ScanSAR visualization (minimal controls).
    - Single adjustable parameter: collection duration (seconds).
    - Platform flies along +X at fixed altitude Z and cross-track Y (set by mid-incidence).
    - Beam is steered in elevation between multiple subswaths (inc_min..inc_max).
    - Shaded ground band shows the total illuminated swath (union of subswaths).
    - Faint subswath centerlines on ground; active subswath shows ellipse footprint.
    - Growing synthetic aperture shows the visited flight track.
    """
    import numpy as np
    import matplotlib.pyplot as plt
    from mpl_toolkits.mplot3d import Axes3D  # noqa: F401
    from mpl_toolkits.mplot3d.art3d import Poly3DCollection
    from ipywidgets import FloatSlider, IntSlider, HBox, VBox, Play, jslink, Button, Layout, Output, Label
    from IPython.display import display, clear_output

    # ---------- Constants (no UI) ----------
    SATELLITE_VELOCITY_KMS = 7.5      # km/s
    ALTITUDE_KM            = 500.0    # km
    INC_MIN_DEG            = 30.0     # deg
    INC_MAX_DEG            = 40.0     # deg
    N_SUBSWATHS            = 3
    DWELL_FRAMES           = 12       # frames per subswath dwell
    BEAM_HALF_ANGLE_DEG    = 0.8      # deg (main-lobe half-angle)
    FPS                    = 12       # frames per second
    MIN_FRAMES             = 60
    MAX_FRAMES             = 600
    K_ELLIPTICITY          = 0.2      # visual elongation in ground range (matches your style)
    ASPECT_SCALE           = (2.2, 1.0, 0.7)  # visually widen X, slightly compress Z

    SUB_COLORS = ['tab:blue','tab:orange','tab:green','tab:red','tab:purple','tab:brown']

    # ---------- Single control ----------
    collection_time = FloatSlider(
        value=12.0, min=4.0, max=60.0, step=2.0,
        description="Collection Time (s)", continuous_update=False
    )
    collection_time.style.description_width = 'initial'

    # Timeline controls
    play = Play(interval=70, value=0, min=0, max=119, step=1)
    frame_slider = IntSlider(value=0, min=0, max=119, step=1, description="Frame", continuous_update=True)
    jslink((play, 'value'), (frame_slider, 'value'))

    reset_btn = Button(description="Reset", layout=Layout(width='90px'))
    recenter_btn = Button(description="Recenter", layout=Layout(width='110px'))

    out = Output()

    # ---------- State ----------
    state = dict(
        x_positions=None, y_positions=None, z_positions=None,
        subswath_inc=None, subswath_Y_abs=None,
        artists={}, fig=None, theta=np.linspace(0, 2*np.pi, 120),
        frames=120
    )

    # ---------- Helpers ----------
    def _compute_frames():
        frames = int(collection_time.value * FPS)
        return int(np.clip(frames, MIN_FRAMES, MAX_FRAMES))

    def _compute_geometry_and_subswaths():
        """
        - Platform path along X at Y = ALT * tan(inc_mid), Z = ALT.
        - Subswath ground centers at Y_abs = ALT * tan(inc) for inc in [INC_MIN_DEG .. INC_MAX_DEG].
        - Shaded swath = union band across all subswaths using widest footprint (max a_rg).
        """
        frames = state['frames']
        aperture_length = SATELLITE_VELOCITY_KMS * collection_time.value  # km
        x_start, x_end = -aperture_length / 2.0, aperture_length / 2.0
        xs = np.linspace(x_start, x_end, frames)

        # Platform Y from mid-incidence
        inc_mid = 0.5 * (INC_MIN_DEG + INC_MAX_DEG)
        y_pos = ALTITUDE_KM * np.tan(np.radians(inc_mid))
        ys = np.full_like(xs, y_pos)
        zs = np.full_like(xs, ALTITUDE_KM)

        state['x_positions'], state['y_positions'], state['z_positions'] = xs, ys, zs

        # Subswath incidence list and absolute ground Y centers (z=0 plane)
        inc_list = np.linspace(INC_MIN_DEG, INC_MAX_DEG, N_SUBSWATHS)
        Y_abs = ALTITUDE_KM * np.tan(np.radians(inc_list)) - y_pos  # absolute ground Y for each subswath
        state['subswath_inc'] = inc_list
        state['subswath_Y_abs'] = Y_abs

        # Determine ellipse semi-axes for min/max incidence — choose max range semi-axis
        def a_rg_for_inc(inc_deg):
            rng = np.sqrt((ALTITUDE_KM * np.tan(np.radians(inc_deg)) - y_pos)**2 + (ALTITUDE_KM**2))
            hw  = np.tan(np.radians(BEAM_HALF_ANGLE_DEG)) * rng
            cosi = max(np.cos(np.radians(inc_deg)), 1e-6)
            return hw / (K_ELLIPTICITY * cosi)

        a_rg_min = a_rg_for_inc(INC_MIN_DEG)
        a_rg_max = a_rg_for_inc(INC_MAX_DEG)
        a_rg_widest = max(a_rg_min, a_rg_max)

        # Shaded illuminated band spans from the lowest subswath center - widest a_rg
        # to the highest subswath center + widest a_rg
        band_lo = float(np.min(Y_abs)) - a_rg_widest
        band_hi = float(np.max(Y_abs)) + a_rg_widest

        # Axes limits with margins: include platform Y, band edges, and zero
        mx = 0.1 * (x_end - x_start + 1.0)
        xlim = (x_start - mx, x_end + mx)

        y_candidates = np.array([band_lo, band_hi, y_pos, 0.0], dtype=float)
        ymin = float(np.min(y_candidates))
        ymax = float(np.max(y_candidates))
        ymargin = 0.1 * (ymax - ymin + 1.0)
        ylim = (ymin - ymargin, ymax + ymargin)

        zlim = (0.0, ALTITUDE_KM * 1.1 + 10.0)

        return (x_start, x_end), xlim, ylim, zlim, band_lo, band_hi

    # ---------- Plot init ----------
    def _init_plot():
        (_, _), xlim, ylim, zlim, band_lo, band_hi = _compute_geometry_and_subswaths()

        fig = plt.figure(figsize=(11, 6))
        ax = fig.add_subplot(111, projection='3d')
        ax.set_title("ScanSAR imaging mode")
        ax.set_xlabel("X (km) — azimuth")
        ax.set_ylabel("Y (km) — ground range")
        ax.set_zlabel("Z (km) — altitude")

        xs = state['x_positions']; ys = state['y_positions']; zs = state['z_positions']
        x_start, x_end = xs[0], xs[-1]

        # --- Shaded illuminated swath (union band on z=0) ---
        swath_poly = [
            (x_start, band_lo, 0.0),
            (x_end,   band_lo, 0.0),
            (x_end,   band_hi, 0.0),
            (x_start, band_hi, 0.0),
        ]
        swath_patch = Poly3DCollection([swath_poly], alpha=0.50, facecolor='lightgreen', edgecolor='none')
        ax.add_collection3d(swath_patch)

        # Static references
        # Full flight path at platform Y,Z
        flight_path = ax.plot(xs, ys, zs, lw=1.0, color='0.8', alpha=0.9, label="Full Flight Path")[0]
        # Ground X-line (Y=0)
        ground_centerline = ax.plot(xs, np.zeros_like(xs), np.zeros_like(xs), lw=1.0, color='0.7', label="Ground Centerline")[0]
        # Subswath guides (absolute ground Y)
        guides = []
        for i, Yc in enumerate(state['subswath_Y_abs']):
            g = ax.plot(xs, np.full_like(xs, Yc), np.zeros_like(xs),
                        lw=0.8, color=SUB_COLORS[i % len(SUB_COLORS)], alpha=0.5)[0]
            guides.append(g)

        # Dynamic artists
        sat_marker = ax.scatter([xs[0]], [ys[0]], [zs[0]],
                                marker='^', s=120, color='orange', edgecolors='k', linewidths=0.8,
                                label='Radar Platform')
        los_line   = ax.plot([], [], [], linestyle='--', lw=1.5, color='red',  label="Beam Center")[0]
        beam_left  = ax.plot([], [], [], linestyle='--', lw=1.0, color='black', label="Beam Edge")[0]
        beam_right = ax.plot([], [], [], linestyle='--', lw=1.0, color='black')[0]

        # Growing synthetic aperture (trajectory so far)
        aperture_line, = ax.plot([], [], [], lw=3.0, color='blue', label="Synthetic Aperture (trajectory)")

        # Instantaneous elliptical footprint on ground (z=0)
        footprint = Poly3DCollection([[(0,0,0)]], alpha=0.7, facecolor='gold', edgecolor='black')
        ax.add_collection3d(footprint)

        # Legend (moved outside to the right)
        ax.legend(
            [sat_marker, aperture_line, los_line, beam_left, footprint, swath_patch, ground_centerline],
            ["Radar Platform", "Aperture (visited track)", "Beam Center", "Beam Edge",
            "Instant Footprint", "Illuminated Swath (shaded)", "Ground Centerline"],
            loc='upper left',
            bbox_to_anchor=(1.05, 1),   # shift to the right outside the axes
            borderaxespad=0.,
            fontsize=9
        )

        # Lock axes & aspect AFTER artists (prevents drift / desqueezes X)
        ax.set_autoscale_on(False)
        ax.set_xlim(*xlim); ax.set_ylim(*ylim); ax.set_zlim(*zlim)
        xspan = xlim[1] - xlim[0]
        yspan = ylim[1] - ylim[0]
        zspan = zlim[1] - zlim[0]
        ax.set_box_aspect((xspan * ASPECT_SCALE[0],
                           yspan * ASPECT_SCALE[1],
                           zspan * ASPECT_SCALE[2]))

        state['fig'] = fig
        state['artists'] = dict(
            ax=ax,
            swath_patch=swath_patch,
            flight_path=flight_path, ground_centerline=ground_centerline,
            subswath_guides=guides,
            sat_marker=sat_marker, los_line=los_line,
            beam_left=beam_left, beam_right=beam_right,
            aperture_line=aperture_line, footprint=footprint
        )

    # ---------- Render helper ----------
    def _render():
        with out:
            clear_output(wait=True)
            display(state['fig'])
            plt.close(state['fig'])

    # ---------- Frame update ----------
    def _update_frame(k):
        if state['x_positions'] is None:
            return

        xs, ys, zs = state['x_positions'], state['y_positions'], state['z_positions']
        xk, yk, zk = xs[k], ys[k], zs[k]

        # Active subswath from dwell cycling
        N = int(N_SUBSWATHS)
        dwell = max(int(DWELL_FRAMES), 1)
        subswath_idx = (k // dwell) % N
        inc_cur = state['subswath_inc'][subswath_idx]
        gp_x = xk
        gp_y = state['subswath_Y_abs'][subswath_idx]  # absolute ground Y
        gp_z = 0.0

        A = state['artists']; ax = A['ax']

        # Move platform marker
        A['sat_marker']._offsets3d = ([xk], [yk], [zk])

        # LOS (platform -> ground intercept at current subswath)
        A['los_line'].set_data([xk, gp_x], [yk, gp_y])
        A['los_line'].set_3d_properties([zk, gp_z])

        # Beam geometry at current dwell
        rng = np.sqrt((yk - gp_y)**2 + (zk - gp_z)**2)  # slant range to current ground point
        a_az = np.tan(np.radians(BEAM_HALF_ANGLE_DEG)) * rng
        cosi = max(np.cos(np.radians(inc_cur)), 1e-6)
        a_rg = a_az / (K_ELLIPTICITY * cosi)

        # Beam edges on ground
        A['beam_left'].set_data([xk, gp_x - a_az], [yk, gp_y])
        A['beam_left'].set_3d_properties([zk, gp_z])
        A['beam_right'].set_data([xk, gp_x + a_az], [yk, gp_y])
        A['beam_right'].set_3d_properties([zk, gp_z])

        # Elliptical footprint (z=0) colored per subswath
        th = state['theta']
        xc = gp_x + a_az * np.cos(th)
        yc = gp_y + a_rg * np.sin(th)
        zc = np.zeros_like(th)
        A['footprint'].set_verts([list(zip(xc, yc, zc))])
        A['footprint'].set_facecolor(SUB_COLORS[subswath_idx % len(SUB_COLORS)])
        A['footprint'].set_alpha(0.35)

        # Growing aperture (visited track)
        A['aperture_line'].set_data(xs[:k+1], ys[:k+1])
        A['aperture_line'].set_3d_properties(zs[:k+1])

        _render()

    # ---------- Rebuild & callbacks ----------
    def _rebuild_scene(*_):
        state['frames'] = _compute_frames()
        play.min = 0; play.max = state['frames'] - 1
        frame_slider.min = 0; frame_slider.max = state['frames'] - 1
        frame_slider.value = 0
        _init_plot()
        _update_frame(0)

    def _on_reset(_):
        frame_slider.value = 0
        _update_frame(0)

    def _on_recenter(_):
        (_, _), xlim, ylim, zlim, *_ = _compute_geometry_and_subswaths()
        ax = state['artists']['ax']
        ax.set_autoscale_on(False)
        ax.set_xlim(*xlim); ax.set_ylim(*ylim); ax.set_zlim(*zlim)
        xspan = xlim[1] - xlim[0]
        yspan = ylim[1] - ylim[0]
        zspan = zlim[1] - zlim[0]
        ax.set_box_aspect((xspan * ASPECT_SCALE[0],
                           yspan * ASPECT_SCALE[1],
                           zspan * ASPECT_SCALE[2]))
        _render()

    frame_slider.observe(lambda ch: _update_frame(ch['new']), names='value')
    reset_btn.on_click(_on_reset)
    recenter_btn.on_click(_on_recenter)
    collection_time.observe(_rebuild_scene, names='value')

    # Build once
    _rebuild_scene()

    # ---------- Layout ----------
    header   = Label("TOPSAR / ScanSAR: subswath cycling with shaded illuminated swath.")
    controls = HBox([collection_time, reset_btn, recenter_btn])
    timeline = HBox([play, frame_slider])

    return VBox([header, controls, timeline, out])


# Build & display
topsar_app = build_topsar_voila_app()
display(topsar_app)